In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [2]:
N = 1000  # size of toy data

def build_linear_dataset(N, p=1, noise_std=0.01):
    X = np.random.rand(N, p)
    # w = 3
    w = 3 * np.ones(p)
    # b = 1
    y = np.matmul(X, w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

In [3]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)

    def forward(self, x):
        return self.linear(x)

regression_model = RegressionModel(1)

In [4]:
loss_fn = torch.nn.MSELoss(size_average=False)
optim = torch.optim.Adam(regression_model.parameters(), lr=0.05)
num_iterations = 1000 if not smoke_test else 2

def main():
    data = build_linear_dataset(N)
    x_data = data[:, :-1]
    y_data = data[:, -1]
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        if (j + 1) % 50 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    # Inspect learned parameters
    print("Learned parameters:")
    for name, param in regression_model.named_parameters():
        print("%s: %.3f" % (name, param.data.numpy()))

if __name__ == '__main__':
    main()

[iteration 0050] loss: 322.7257
[iteration 0100] loss: 172.3712
[iteration 0150] loss: 106.8605
[iteration 0200] loss: 60.2285
[iteration 0250] loss: 31.0144
[iteration 0300] loss: 14.6806
[iteration 0350] loss: 6.4274
[iteration 0400] loss: 2.6312
[iteration 0450] loss: 1.0346
[iteration 0500] loss: 0.4191
[iteration 0550] loss: 0.2011
[iteration 0600] loss: 0.1303
[iteration 0650] loss: 0.1091
[iteration 0700] loss: 0.1033
[iteration 0750] loss: 0.1018
[iteration 0800] loss: 0.1015
[iteration 0850] loss: 0.1014
[iteration 0900] loss: 0.1014
[iteration 0950] loss: 0.1014
[iteration 1000] loss: 0.1014
Learned parameters:
linear.weight: 2.999
linear.bias: 1.001
